# Program Execution Metadata

In [ ]:
# Max execution time of a solver before terminating it (in seconds)
# Set to -1 for unlimited execution time
TIME_LIMIT = 60

# Imports

In [ ]:
# Imports
import time
import heapq

# S-Puzzle Class Definition

In [ ]:
# Class definition for S-Puzzle
class S_Puzzle:
    """
    A class to represent and manipulate an S-Puzzle.
    """
    
    def __init__(self, initial_state):
        """
        Create an S-Puzzle with passed initial_state.
        initial_state should be a n x n tuple of unique integers from 1 to n^2
        """
        
        # Assume input initial_state is valid
        self.__size = len(initial_state)
        self.set_state(initial_state)
    
    def get_size(self):
        """
        Return the size of the puzzle.
        
        For an n x n puzzle, return n.
        """
        return self.__size
    
    def get_state(self):
        """
        Return the current state of the puzzle as an n x n tuple.
        """
        
        return self.__state
    
    def set_state(self, state):
        """
        Set state of puzzle.
        """
        
        # Assume input state is valid, and same size as self.__size
        self.__state = state
        
    def get_current_state_children(self):
        """
        Return an array of possible next moves from current state.
        """
        
        children = []
        n = self.__size
        state = self.__state
        
        # Horizontal swaps
        for row in range(n):
            row_tuple = self.__state[row]
            for col in range(n-1):
                new_row = row_tuple[:col] + (row_tuple[col+1],) + (row_tuple[col],) + row_tuple[col+2:]
                new_tuple = state[:row] + (new_row,) + state[row+1:]
                children.append(new_tuple)
        
        # Vertical swaps
        for row in range(n-1):
            row1 = state[row]
            row2 = state[row+1]
            for col in range(n):
                new_row1 = row1[:col] + (row2[col],) + row1[col+1:]
                new_row2 = row2[:col] + (row1[col],) + row2[col+1:]
                new_tuple = state[:row] + (new_row1,) + (new_row2,) + state[row+2:]
                children.append(new_tuple)
        
        return children
    
    def is_current_state_goal(self):
        """
        Returns true if the current state is the goal state.
        """
        
        n = self.__size
        prev = 0
        for row in range(n):
            for col in range(n):
                if self.__state[row][col] != prev + 1:
                    return False
                prev += 1
        return True

# Solver Algorithms Class Definitions

In [ ]:
# DFS Solver class definition
class DFS_Solver:
    """
    A class to solve S-Puzzles using DFS.
    """
    
    def solve(self, puzzle):
        """
        Solve the passed puzzle using DFS.
        
        Returns success (boolean), solution_stack (list representing the solution from the initial state to goal state),
        search_stack (list of the full space the DFS searched to find the solution), and execution time (in seconds).
        """
        
        # Record starting time
        self.__start_time = time.time()
        
        self.__seen_set = set()
        self.__solution_stack = []
        self.__search_stack = []
        success = self.__solve_dfs(puzzle)
        
        return success, self.__solution_stack, self.__search_stack, (time.time() - self.__start_time)
        
    def __solve_dfs(self, puzzle):
        """
        Perform DFS on puzzle.
        Populates the search_stack, solution_stack, and seen_set during the search.
        Returns True if current state is goal (to notify the previous calls to stop DFS), False otherwise.
        """
        
        # Stop DFS if we have exceeded max execution time
        if TIME_LIMIT != -1 and (time.time() - self.__start_time) >= TIME_LIMIT:
            return False
        
        state = puzzle.get_state()
        
        # Add current state to search_stack, solution_stack, and seen_set
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        # Return True if current state is goal to notify previous calls to stop
        if puzzle.is_current_state_goal():
            return True
        
        # Get children of current state and iterate
        children = puzzle.get_current_state_children()
        
        for child in children:
            if child in self.__seen_set:
                continue
            
            puzzle.set_state(child)
            
            # Recursively solve puzzle
            success = self.__solve_dfs(puzzle)
            
            # If solution was found through this node, return True to stop previous calls
            if success:
                return True
            
            # Append "Backtrack" to search_stack to show backtracking
            self.__search_stack.append("Backtrack\n")
        
        # Pop current state from solution_stack since we did not find a solution through this state
        self.__solution_stack.pop()
        return False

In [ ]:
# Iterative Deepening Solver class definition
class Iterative_Deepening_Solver:
    """
    A class to solve S-Puzzles using Iterative Deepening (DFS with increasing depth limits).
    """
    
    def solve(self, puzzle):
        """
        Solve the passed puzzle using Iterative Deepening (DFS with increasing depth limits).
        
        Returns success (boolean), solution_stack (list representing the solution from the initial state to goal state),
        search_stack (list of the full space the DFS searched to find the solution per iteration),
        and execution time (in seconds).
        """
        
        # Record starting time
        self.__start_time = time.time()
        
        initial_state = puzzle.get_state()
        
        # Create a global_search_stack list that will contain lists of the search space of each iteration
        global_search_stack = []
        
        limit = 0
        while True:
            # Reset puzzle state to initial_state
            puzzle.set_state(initial_state)
            
            self.__seen_set = set()
            self.__solution_stack = []
            self.__search_stack = []
            
            # max_depth_seen will contain the max depth the DFS reaches
            # If max_depth_seen is lower than our current depth limit,
            # then we stop because we have searched the WHOLE state space
            self.__max_depth_seen = 0
            
            # Perform a DFS with depth limit of "limit"
            success = self.__solve_dfs(puzzle, 0, limit)
            
            # Append search_stack for current iteration go global_search_stack
            global_search_stack.append(self.__search_stack)
            
            # If DFS found a solution or max_depth_seen is lower than current depth limit (full state space searched),
            # Then stop
            if success or self.__max_depth_seen < limit:
                break
            
            # Increase limit because no solution was found with current limit
            limit += 1
        
        return success, self.__solution_stack, global_search_stack, (time.time() - self.__start_time)
        
    def __solve_dfs(self, puzzle, current_depth, limit):
        """
        Perform DFS on puzzle with depth limit of "limit".
        Populates the search_stack, solution_stack, and seen_set during the search.
        Returns True if current state is goal (to notify the previous calls to stop DFS), False otherwise.
        """
        
        # Stop DFS if we have exceeded max execution time
        if TIME_LIMIT != -1 and (time.time() - self.__start_time) >= TIME_LIMIT:
            return False
        
        # Update max_depth_seen if necessary
        if current_depth > self.__max_depth_seen:
            self.__max_depth_seen = current_depth
        
        state = puzzle.get_state()
        
        # Add current state to search_stack, solution_stack, and seen_set
        self.__search_stack.append(state)
        self.__solution_stack.append(state)
        self.__seen_set.add(state)
        
        # Return True if current state is goal to notify previous calls to stop
        if puzzle.is_current_state_goal():
            return True
        
        # Calculate depth for next calls
        new_depth = current_depth + 1
        
        # If depth for next calls is within the limits, continue DFS
        if new_depth <= limit:
            # Get children of current state and iterate through children
            children = puzzle.get_current_state_children()
            for child in children:
                if child in self.__seen_set:
                    continue

                puzzle.set_state(child)
                
                # Recursively solve puzzle
                success = self.__solve_dfs(puzzle, current_depth + 1, limit)

                # If solution was found through this node, return True to stop previous calls
                if success:
                    return True
                
                # Append "Backtrack" to search_stack to show backtracking
                self.__search_stack.append("Backtrack\n")
        
        # Pop current state from solution_stack since we did not find a solution through this state
        self.__solution_stack.pop()
        return False

In [ ]:
# A* Solver Class Definition
class A_Star_Solver:
    """
    A class to solve S-Puzzles using A* with a specified heuristic function
    """
    
    def __init__(self, heuristic):
        """
        Create an A* Solver with passed heuristic function.
        """
        
        self.__heuristic = heuristic
    
    def solve(self, puzzle):
        """
        Solve the passed puzzle using A*.
        
        Returns success (boolean), solution_stack (list representing the solution from the initial state to goal state),
        search_stack (list of the full space the DFS searched to find the solution), and execution time (in seconds).
        """
        
        
        # Create min heap
        # Elements of heap will be tuples of the form:
        # ( cost + heuristic, cost, parent, state )
        # Where heuristic should be an estimate of the cost to reach goal from state
        # By default, heappush and heappop will compare by first element of tuple
        heap = []
        
        # Create parents dictionary, mapping a state to its parent state
        parents = dict()
        
        search = []
        seen = set()
        success = False
        
        state = puzzle.get_state()
        
        # Push the initial state with priority 0, 0 cost, and no parent state
        heapq.heappush(heap, (0, 0, None, state))
        
        # Record starting time
        start_time = time.time()
        
        # While there are states left to explore, continue A* algorithm
        while heap:
            # Stop if we have exceeded max execution time
            if TIME_LIMIT != -1 and (time.time() - start_time) >= TIME_LIMIT:
                break
            
            # Get priority, cost, parent, and state of min element of heap
            priority, cost, parent, state = heapq.heappop(heap)
            
            # Ignore this state if it has already been seen.
            # If it has already been seen, it means that this current state can be reached
            # with a smaller priority/cost than what it is currently.
            # This can happen if multiple parent states can reach this state,
            # but this state is not explored until later on (i.e. this state has been pushed multiple time
            # to the heap before being explored).
            if state in seen:
                continue
            
            puzzle.set_state(state)
            
            # Add current state to search stack and seen set
            search.append(state)
            seen.add(state)
            
            # Map current state to its parent
            parents[state] = parent
            
            # If current state is goal state,
            # set success flag to True and break out of the loop
            if puzzle.is_current_state_goal():
                success = True
                break
            
            # Get children of current state
            children = puzzle.get_current_state_children()
            
            # Calculate cost for children states
            new_cost = cost + 1
            
            for child in children:
                # Do not add child state to heap if it was seen
                if child in seen:
                    continue
                
                # Add child state to heap with priority = new_cost + heuristic(child_state),
                # cost = new_cost, and parent_state = state
                heapq.heappush(heap, (new_cost + self.__heuristic(child), new_cost, state, child))
        
        # Create an empty solution list
        solution = []
        
        # If success flag is true, populate solution list
        if success:
            # Keep adding state to solution list, and set state to its parent state
            # This will find the path the algorithm took to get to the goal state
            while state is not None:
                solution.append(state)
                state = parents[state]
            # Reverse solution list to sort list from initial state to goal
            solution.reverse()
            
        return success, solution, search, (time.time() - start_time)
                

# Helper Functions

In [ ]:
# Helper functions

# Function to nicely display a puzzle state
def display_puzzle_state(state):
    """
    Prints a puzzle state in a human-readable way.
    """
    
    size = len(state)
    for i in range(size):
        print(state[i])
    print()

# Function to display results of solvers
def print_results(success, solution_path, search_path, execution_time):
    """
    Prints the results of a Solver in a human-readable way.
    """
    
    print('Execution time:', execution_time, 's')
    print('Success:', success, "(No solution found)" if not success else "")
    print("=================================")
    print('Solution path (Cost: {}):'.format(len(solution_path) - 1))
    for state in solution_path:
        display_puzzle_state(state)
    print("=================================")
    print('Search path:')
    
    for i in range(len(search_path)):
        item = search_path[i]
        if isinstance(item, str):
            # If element is simply a string,
            # Print it directly
            print(item)
        elif isinstance(item, tuple):
            # If element is a tuple (i.e. state),
            # Display it with helper function
            display_puzzle_state(item)
        elif isinstance(item, list):
            # In iterative deepening, the elements of search_path are lists,
            # Each representing the search space of their corresponding depth limit
            print("================")
            print('Limit:', i)
            for state in item:
                if isinstance(state, str):
                    print(state)
                else:
                    display_puzzle_state(state)
                

# Heuristic Functions Definitions

In [ ]:
# Heuristic definitions

def h1(state):
    """
    (NOT ADMISSIBLE)
    Returns number of tiles out of place in passed state.
    """

    # Admissibility counter-example:
    # (2,1,3)
    # (4,5,6)
    # (7,8,9)
    # Then h1(state) = 2, when real_cost = 1
    # Hence, non-admissible
    
    out_of_place = 0
    current_count = 1
    size = len(state)
    for row in range(size):
        for col in range(size):
            if state[row][col] != current_count:
                out_of_place += 1
            current_count += 1
    return out_of_place

def h2(state):
    """
    (NOT ADMISSIBLE)
    Returns the Manhattan distance of passed state (sum of all the distances by which tiles are out of place).
    """
    
    # Admissibility counter-example:
    # (2,1,3)
    # (4,5,6)
    # (7,8,9)
    # Then h2(state) = 2, when real_cost = 1
    # Hence, non-admissible
    
    sum = 0
    size = len(state)
    for row in range(size):
        for col in range(size):
            num = state[row][col]
            real_row = (num-1) // size
            real_col = (num-1) % size
            manhattan_distance = abs(row-real_row) + abs(col-real_col)
            sum += manhattan_distance
    return sum

def h3(state):
    """
    (NOT ADMISSIBLE)
    Returns the number of inversions in the passed state.
    """
    
    # Admissibility counter-example:
    # (4,2,3)
    # (1,5,6)
    # (7,8,9)
    # Then h3(state) = 5, when real_cost = 1
    # Hence, non-admissible
    
    # sum of permutation inversions
    # merge sort can find number of permutation inversions in O(nlogn)
    # but we do it naively (i.e. O(n^2)) because input is small
    # and naive performs better for smaller inputs
    total_inversions = 0
    size = len(state)
    cap = size*size
    for i in range(cap):
        row1 = i // size
        col1 = i % size
        for j in range(i+1, cap):
            row2 = j // size
            col2 = j % size
            if state[row1][col1] > state[row2][col2]:
                total_inversions += 1
    return total_inversions

def h4(state):
    """
    (Admissibility unknown)
    Returns a modified version of Manhattan distance of passed state
    (sum of all the distances by which tiles are out of place).
    """
    
    # BEST SO FAR!
    
    # The reason why Manhattan distance is not admissible is because of cases such as:
    # (2,1,3)
    # (4,5,6)
    # (7,8,9)
    # In this case, Manhattan distance would be 2 because 2 is misplaced by 1 and 1 is misplaced by one.
    # However, this overestimates the actual cost because by swapping 1 and 2, we actually move BOTH
    # to their correct place in one single move
    #
    # Hence, the reasoning behind this heuristic function is that when we find a misplaced number (let's say on tile X),
    # we look at the tile where the number is actually supposed to be (let's say tile Y). If the number of tile Y is
    # supposed to be on tile X, then we substract 1 from the Manhattan sum.
    #
    # In particular, in the example state above, this heuristic function would return an estimated cost of 1.
    # The Manhattan distance of tile 2 is 1, and the Manhattan distance of tile 1 is 1. The sum is hence 2.
    # However, since tile 1 and tile 2 belong in each other's places, we substract 1, giving a total of 1.
    # This is equal to the real cost of 1.
    #
    # The reason why this "trick" would also work when "swapped" tiles are not directly next to each other can be seen
    # in the following example:
    # (3,2,1)
    # (4,5,6)
    # (7,8,9)
    # In this case, the Manhattan distance of tile 3 is 2, and the Manhattan distance of tile 1 is also 2.
    # The sum is then 4. However, 3 and 1 are in each other's places, so we substract 1, giving a total of 3.
    # This is equal to the real cost of 3.
    # Why this works is because even though they are not directly next to each other (i.e. we cannot swap them both
    # in one move like in the first example above), if we theoretically moved 3 to its spot (i.e. swap 3 and 2, then
    # swap 3 and 1), we would actually move tile 1 closer towards its real place, hence decreasing the amount of
    # necessary moves by 1 (i.e. decrease cost by 1)!
    # This supports the reasoning behind the approximation of this heuristic function.
    
    sum = 0
    size = len(state)
    current = 1
    for row in range(size):
        for col in range(size):
            num = state[row][col]
            real_row = (num-1) // size
            real_col = (num-1) % size
            manhattan_distance = abs(row-real_row) + abs(col-real_col)
            sum += manhattan_distance
            
            if num > current:
                if state[real_row][real_col] == current:
                    sum -= 1
            
            current += 1
    return sum

# Solvers Executions

In [ ]:
# DFS Solver

# Prone to stack overflow because the algorithm searches naively
# and might not find solution even after 1000 recursive calls,
# which will throw a RecursionError (max recursion depth exceeded)


puzzle = S_Puzzle( ((3,4), (1,2)) )
dfs_solver = DFS_Solver()

print_results(*dfs_solver.solve(puzzle))

In [ ]:
# Iterative Deepening Solver

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
id_solver = Iterative_Deepening_Solver()

print_results(*id_solver.solve(puzzle))

In [ ]:
# A* solver (Heuristic h1)

puzzle = S_Puzzle( ((6, 1, 2), (7, 8, 3), (5, 4, 9)) )
a_star = A_Star_Solver(heuristic=h1)

print_results(*a_star.solve(puzzle))

In [ ]:
# A* solver (Heuristic h2)

puzzle = S_Puzzle( ((14,1,7,16),(8,5,2,6),(9,13,15,4),(3,10,11,12)) )
a_star = A_Star_Solver(heuristic=h2)

print_results(*a_star.solve(puzzle))

In [ ]:
# A* solver (Heuristic h3)

puzzle = S_Puzzle( ((14,1,7,16),(8,5,2,6),(9,13,15,4),(3,10,11,12)) )
a_star = A_Star_Solver(heuristic=h3)

print_results(*a_star.solve(puzzle))

In [ ]:
# A* solver (Heuristic h4)

puzzle = S_Puzzle( ((14,1,7,16),(8,5,2,6),(9,13,15,4),(3,10,11,12)) )
a_star = A_Star_Solver(heuristic=h4)

print_results(*a_star.solve(puzzle))